In [1]:
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

from sklearn.utils import class_weight
from pyvi import ViTokenizer
from sklearn.metrics import confusion_matrix,classification_report
from collections import Counter
from sklearn.utils import resample
from utils import load_checkpoint

In [2]:
from model import AttentionModel
from dataset import TextDataset

In [3]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

# Load data

In [4]:
# data_path = '../data'
# model_path = '../model'

In [5]:
df_ = pd.read_csv(r'E:\NLP\Vietnamese-Text-Classification-master\data\test.csv')

In [5]:
test_data = df_.sample(n=5000, random_state=42)  # Lấy ngẫu nhiên 1000 bản ghi
train_data = df_.drop(test_data.index)  # Loại bỏ các bản ghi test khỏi tập train

# In thông tin về kích thước của tập train và test
print(f'Size of training set: {len(train_data)}')
print(f'Size of test set: {len(test_data)}')

# Lưu dữ liệu vào các tệp CSV riêng biệt
# train_data.to_csv('train.csv', index=False)
# test_data.to_csv('test.csv', index=False)

NameError: name 'df_' is not defined

In [7]:
train_data.head(10)

NameError: name 'train_data' is not defined

In [ ]:
test_data.head(10)

In [ ]:
drop_columns=['Date','Author(s)','Tags']
train_data = train_data.drop(columns=drop_columns)
test_data =test_data.drop(columns=drop_columns)
train_data.head(10)

In [ ]:
unique_values = train_data['Category'].unique()
unique_values

In [ ]:
def display_missing(df,features_columns):
    n_rows=df.shape[0]
    for col in features_columns:
        missing_count=df[col].isnull().sum()
        if missing_count>0:
            print(f"{col} has {missing_count} missing values")
column_names = train_data.columns
print("Train:",display_missing(train_data,column_names))
print("Test:",display_missing(test_data,column_names))

In [ ]:
train_data = train_data.dropna(subset=['Contents'])
train_data = train_data.dropna(subset=['Summary'])
test_data = test_data.dropna(subset=['Contents'])
test_data = test_data.dropna(subset=['Summary'])
print(train_data)
print(test_data)

In [ ]:
print("Train:",display_missing(train_data,column_names))
print("Test:",display_missing(test_data,column_names))

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
train_data['Category'] = label_encoder.fit_transform(train_data['Category'])
test_data['Category'] = label_encoder.fit_transform(test_data['Category'])

In [ ]:
unique_values = train_data['Category'].unique()
unique_values

In [ ]:
train_data['Category'].value_counts()

In [ ]:
train_data.head(10)

In [ ]:
train_data['Contents'] = pd.concat([train_data['Contents'], train_data['Summary']], axis=1)
train_data = df.drop('Summary', axis=1)

In [ ]:
train_data

In [ ]:
train_data.columns

In [ ]:
question  = train_data

In [ ]:
question['length'] = [len(item) for item in list(question['Contents'])]
question['num_word'] = [len(item.split(' ')) for item in list(question['Contents'])]

In [ ]:
np.mean(question['num_word'])

In [ ]:
plt.hist(list(question['num_word']), bins = 1000)

In [ ]:
def tokenize(text):
    list_token = ViTokenizer.tokenize(text)
    return list_token.split(' ')

In [ ]:
counts = Counter()
for index, row in question.iterrows():
    counts.update(tokenize(row['Contents']))

In [ ]:
#deleting infrequent words
print("num_words before:",len(counts.keys()))
for word in list(counts):
    if counts[word] < 2:
        del counts[word]
print("num_words after:",len(counts.keys()))

In [ ]:
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)

In [ ]:
def encode_sentence(text, vocab2index, N=75):
    tokenized = tokenize(text)
    encoded = np.zeros(N, dtype=int)
    enc1 = np.array([vocab2index.get(word, vocab2index["UNK"]) for word in tokenized])
#     print(len(enc1))
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
#     print(len(encoded))
    return [encoded]
#     return encoded, length

In [ ]:
N = 22
question['encoded'] = question['Contents'].apply(lambda x: np.array(encode_sentence(x,vocab2index,N)))
question.head()

In [ ]:
question.to_csv('/kaggle/working/train_tokenized.csv', index=False)

# Test

In [ ]:
# load_model

In [ ]:
# filename = os.path.join(model_path,'model_intent.pth')
# joblib.dump(load_model, filename)
# # with open('vectorizer.pickle', 'wb') as handle:
# #     pickle.dump(vectorizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# import requests
# url = 'https://api-intent.herokuapp.com/predict'
# pred = requests.post(url,json={'message':'ad cho em hỏi chương trình tiên tiến với chất lượng cao khác nhau thế nào ạ'})
# print(pred.json())


In [ ]:
# for x,y in val_dl:
#     print(x.shape)

In [ ]:
# vocab2index